In [3]:
import tensorflow as tf
K = tf.keras

with open('warpeace_input.txt','r') as f:
    data = f.read()

uniq_char = set(data)

idx2char = dict()
char2idx = dict()

for i,x in enumerate(uniq_char):
    idx2char[i] = x
    char2idx[x] = i

print(char2idx)
print(idx2char)

{'\n': 0, '!': 1, '6': 2, '»': 3, 'Y': 4, 'X': 5, 'j': 6, 'ª': 7, 'u': 8, 'c': 9, '?': 10, 'r': 11, 'V': 12, '©': 13, 'h': 14, 'f': 15, 'J': 16, 'a': 17, 'm': 18, '9': 19, '1': 20, '7': 21, 'w': 22, ':': 23, '4': 24, 'Q': 25, 'B': 26, '"': 27, ',': 28, 'q': 29, '2': 30, '¿': 31, 'v': 32, 'P': 33, ')': 34, '*': 35, 'A': 36, 'F': 37, 'C': 38, 'y': 39, '3': 40, ' ': 41, 'L': 42, 'Z': 43, 'g': 44, 'l': 45, 'H': 46, "'": 47, 'ï': 48, 'N': 49, 'd': 50, 'p': 51, 'Ã': 52, '8': 53, 'b': 54, '\xa0': 55, 't': 56, 'K': 57, 'R': 58, 'T': 59, 'O': 60, '-': 61, 'e': 62, 'x': 63, 'W': 64, 'o': 65, 'D': 66, 'i': 67, 's': 68, 'I': 69, '=': 70, '.': 71, '(': 72, 'z': 73, '5': 74, '0': 75, 'n': 76, '¤': 77, ';': 78, '/': 79, 'U': 80, 'k': 81, 'M': 82, 'G': 83, 'S': 84, 'E': 85}
{0: '\n', 1: '!', 2: '6', 3: '»', 4: 'Y', 5: 'X', 6: 'j', 7: 'ª', 8: 'u', 9: 'c', 10: '?', 11: 'r', 12: 'V', 13: '©', 14: 'h', 15: 'f', 16: 'J', 17: 'a', 18: 'm', 19: '9', 20: '1', 21: '7', 22: 'w', 23: ':', 24: '4', 25: 'Q', 26: '

In [4]:
VOCAB_SIZE = len(char2idx)
print(VOCAB_SIZE)

86


In [5]:
# Hyper-parameter: size of context window.
SEQ_LEN = 32
STEP = 3

In [6]:
input_sequence = []
output_char = []

for start in range(0,len(data)-SEQ_LEN,STEP):
    input_sequence.append(data[start:start+SEQ_LEN])
    output_char.append(data[start+SEQ_LEN])
print('# Of sequences',len(input_sequence))

# Of sequences 1065400


In [7]:
NB_SEQ = len(input_sequence)

In [8]:
import numpy as np
# With the default dtype=float these arrays will cause OOM.
X_one_hot = np.zeros([NB_SEQ,SEQ_LEN,VOCAB_SIZE],dtype=np.int8)
y_one_hot = np.zeros([NB_SEQ,VOCAB_SIZE],dtype=np.int8)

In [9]:
for i in range(NB_SEQ):
    # Iterate over all sequences.
    # Convert each sequence to indices in the vocabulary.
    seq_as_char = input_sequence[i]
    seq_as_idx  = [char2idx[c] for c in seq_as_char]
    # Convert each character in the sequence to one-hot representation.
    for seq_id,char_idx in enumerate(seq_as_idx):
        X_one_hot[i,seq_id,char_idx] = 1
    y_one_hot[i,char2idx[output_char[i]]] = 1

In [10]:
HIDDEN_STATE_SZ = 512
NB_LAYERS = 3

In [22]:


model = K.models.Sequential()
model.add(K.layers.LSTM(HIDDEN_STATE_SZ, input_shape=(SEQ_LEN, VOCAB_SIZE), return_sequences=True))
for i in range(NB_LAYERS - 2):
    model.add(K.layers.LSTM(HIDDEN_STATE_SZ, return_sequences=True))
model.add(K.layers.LSTM(HIDDEN_STATE_SZ))
model.add(K.layers.Dense(VOCAB_SIZE))
model.add(K.layers.Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [23]:
model.fit(X_one_hot,y_one_hot,batch_size=128)

Epoch 1/1
  22016/1065400 [..............................] - ETA: 23:10:56 - loss: 3.0816

KeyboardInterrupt: 

In [ ]:
# Used from Keras LSTM example:
#https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - SEQ_LEN - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = data[start_index: start_index + SEQ_LEN]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, SEQ_LEN, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2idx[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = idx2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(X_one_hot,y_one_hot,batch_size=128, epochs=5, callbacks=[print_callback])